In [2]:
import pandas as pd

duration_df = pd.read_excel('./Study tracker - JS.xlsx')

duration_df.head()

,Month,Date,Task,Start,End,Duration (hh:mm)
0,January,2024-01-18,FCC - form,09:40:00,12:00:00,02:20:00
1,January,2024-01-18,FCC - form,13:40:00,15:05:00,01:25:00
2,January,2024-01-18,Reading - The Odin Project,15:15:00,16:15:00,01:00:00
3,January,2024-01-18,FCC - Project (stuck),19:20:00,19:50:00,00:30:00
4,January,2024-01-18,Youtube tutorial,19:55:00,20:20:00,00:25:00


In [3]:
duration_df.columns

Index(['Month', 'Date', 'Task', 'Start', 'End', 'Duration (hh:mm)'], dtype='object')

In [4]:
# Step 1: Convert all values to strings and handle any unusual formats
duration_df['Duration (hh:mm)'] = duration_df['Duration (hh:mm)'].astype(str)

# Step 2: Remove any rows with multiple negative signs, unexpected characters, or incorrect formats
# Here we filter for only valid time formats: e.g., "HH:MM:SS" or "HH:MM"
import re
valid_time_pattern = re.compile(r'^(?:-)?\d{1,2}:\d{2}(?::\d{2})?$')  # Matches "HH:MM" or "HH:MM:SS" formats
duration_df = duration_df[duration_df['Duration (hh:mm)'].apply(lambda x: bool(valid_time_pattern.match(x)))]

# Step 3: Convert valid time strings to timedelta
duration_df['Duration (hh:mm)'] = pd.to_timedelta(duration_df['Duration (hh:mm)'], errors='coerce')

# Step 4: Convert timedelta to total minutes
duration_df['Duration_minutes'] = duration_df['Duration (hh:mm)'].dt.total_seconds() / 60

print(duration_df)


       Month       Date                          Task     Start       End  \
0    January 2024-01-18                    FCC - form  09:40:00  12:00:00   
1    January 2024-01-18                    FCC - form  13:40:00  15:05:00   
2    January 2024-01-18    Reading - The Odin Project  15:15:00  16:15:00   
3    January 2024-01-18         FCC - Project (stuck)  19:20:00  19:50:00   
4    January 2024-01-18              Youtube tutorial  19:55:00  20:20:00   
..       ...        ...                           ...       ...       ...   
548     June 2024-06-21              Python - reading  03:05:00  03:25:00   
549     June 2024-06-21             Portfolio - No. 2  03:30:00  03:45:00   
550     June 2024-06-21  Potfolio - add functionality  03:55:00  06:35:00   
551     June 2024-06-21            Update error - idk  09:10:00  10:05:00   
552     June 2024-06-22                  Update error  06:25:00  08:45:00   

    Duration (hh:mm)  Duration_minutes  
0    0 days 02:20:00             1

In [5]:
duration_df.columns

Index(['Month', 'Date', 'Task', 'Start', 'End', 'Duration (hh:mm)',
       'Duration_minutes'],
      dtype='object')

In [6]:
duration_df['Date'] = pd.to_datetime(duration_df['Date'], unit='ms').dt.strftime('%Y-%m-%d')

duration_2 = duration_df.drop(columns=['Month', 'Duration (hh:mm)'])

# duration_2.to_json('tracker.json', orient='records', lines=False)

print(duration_2)

           Date                          Task     Start       End  \
0    2024-01-18                    FCC - form  09:40:00  12:00:00   
1    2024-01-18                    FCC - form  13:40:00  15:05:00   
2    2024-01-18    Reading - The Odin Project  15:15:00  16:15:00   
3    2024-01-18         FCC - Project (stuck)  19:20:00  19:50:00   
4    2024-01-18              Youtube tutorial  19:55:00  20:20:00   
..          ...                           ...       ...       ...   
548  2024-06-21              Python - reading  03:05:00  03:25:00   
549  2024-06-21             Portfolio - No. 2  03:30:00  03:45:00   
550  2024-06-21  Potfolio - add functionality  03:55:00  06:35:00   
551  2024-06-21            Update error - idk  09:10:00  10:05:00   
552  2024-06-22                  Update error  06:25:00  08:45:00   

     Duration_minutes  
0               140.0  
1                85.0  
2                60.0  
3                30.0  
4                25.0  
..                ...  
548

In [7]:
duration_2.columns

Index(['Date', 'Task', 'Start', 'End', 'Duration_minutes'], dtype='object')

**EXPORT JSON FROM PANDAS - UPLOAD TO SUPABASE**
- pip install supabase

In [14]:
import json 
import os 
from supabase import create_client, Client
from dotenv import load_dotenv

load_dotenv()

supabase_url = os.getenv("SUPABASE_URL")
supabase_key = os.getenv("SUPABASE_KEY")

#initialize supabase client
supabase: Client = create_client(supabase_url, supabase_key)

#load json data
with open("tracker.json", "r") as file:
    data = json.load(file)
    
#convert duraiton to integers
for record in data:
    record['Duration_minutes'] = round(record['Duration_minutes'])
    
#insert data into supabase table
# for record in data:
#     response = supabase.table("tracker").insert(record).execute()
#     print(response)
